In [1]:
import pandas as pd
import arff
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
with open('dane.arff', 'r') as f:
    data = arff.load(f)
attributes = [attr[0] for attr in data['attributes']]
data_rows = data['data']
df = pd.DataFrame(data_rows, columns=attributes)

VARIABLE DESCRIPTIONS
- Pclass Passenger Class (1 = 1st; 2 = 2nd; 3 = 3rd)
- survival Survival (0 = No; 1 = Yes)
- name Name
- sex Sex
- age Age
- sibsp Number of Siblings/Spouses Aboard
- parch Number of Parents/Children Aboard
- ticket Ticket Number
- fare Passenger Fare (British pound)
- cabin Cabin
- embarked Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)
- boat Lifeboat
- body Body Identification Number
- home.dest Home/Destination

In [3]:
df.head(10)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1.0,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0.0,0.0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1.0,1,"Allison, Master. Hudson Trevor",male,0.9167,1.0,2.0,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"
2,1.0,0,"Allison, Miss. Helen Loraine",female,2.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,NaN,"Montreal, PQ / Chesterville, ON"
3,1.0,0,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,135.0,"Montreal, PQ / Chesterville, ON"
4,1.0,0,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1.0,2.0,113781,151.5500,C22 C26,S,None,NaN,"Montreal, PQ / Chesterville, ON"
5,1.0,1,"Anderson, Mr. Harry",male,48.0000,0.0,0.0,19952,26.5500,E12,S,3,NaN,"New York, NY"
6,1.0,1,"Andrews, Miss. Kornelia Theodosia",female,63.0000,1.0,0.0,13502,77.9583,D7,S,10,NaN,"Hudson, NY"
7,1.0,0,"Andrews, Mr. Thomas Jr",male,39.0000,0.0,0.0,112050,0.0000,A36,S,None,NaN,"Belfast, NI"
8,1.0,1,"Appleton, Mrs. Edward Dale (Charlotte Lamson)",female,53.0000,2.0,0.0,11769,51.4792,C101,S,D,NaN,"Bayside, Queens, NY"
9,1.0,0,"Artagaveytia, Mr. Ramon",male,71.0000,0.0,0.0,PC 17609,49.5042,None,C,None,22.0,"Montevideo, Uruguay"


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 14 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   pclass     1309 non-null   float64
 1   survived   1309 non-null   object 
 2   name       1309 non-null   object 
 3   sex        1309 non-null   object 
 4   age        1046 non-null   float64
 5   sibsp      1309 non-null   float64
 6   parch      1309 non-null   float64
 7   ticket     1309 non-null   object 
 8   fare       1308 non-null   float64
 9   cabin      295 non-null    object 
 10  embarked   1307 non-null   object 
 11  boat       486 non-null    object 
 12  body       121 non-null    float64
 13  home.dest  745 non-null    object 
dtypes: float64(6), object(8)
memory usage: 143.3+ KB


Kolumny pclass, sibsp, parch, body też uznam jako zmienne jakościowe tak więc zmieniam ich typ na string, żeby w dalszej analizie zakwalifikowwały się jako 'categorical_columns'

In [5]:
df['pclass'] = df['pclass'].astype('str')
df['sibsp'] = df['sibsp'].astype('str')
df['parch'] = df['parch'].astype('str')
df['body'] = df['body'].astype('str')

1. Sprawdź liczebność poszczególnych etykiet dla danych zmiennych jakościowych

In [6]:
categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
for column_name in categorical_columns:
    unique_values = len(df[f'{column_name}'].unique())
    # Uznałem 10 jako granicę mocy zbioru pomiędzy wysoką a niską
    cardinality = 'wysoka' if unique_values > 10 else 'niska'
    print(f'Liczba etykiet zmiennej {column_name}: {unique_values}, Moc zbioru: {cardinality}')
    

Liczba etykiet zmiennej pclass: 3, Moc zbioru: niska
Liczba etykiet zmiennej survived: 2, Moc zbioru: niska
Liczba etykiet zmiennej name: 1307, Moc zbioru: wysoka
Liczba etykiet zmiennej sex: 2, Moc zbioru: niska
Liczba etykiet zmiennej sibsp: 7, Moc zbioru: niska
Liczba etykiet zmiennej parch: 8, Moc zbioru: niska
Liczba etykiet zmiennej ticket: 929, Moc zbioru: wysoka
Liczba etykiet zmiennej cabin: 187, Moc zbioru: wysoka
Liczba etykiet zmiennej embarked: 4, Moc zbioru: niska
Liczba etykiet zmiennej boat: 28, Moc zbioru: wysoka
Liczba etykiet zmiennej body: 122, Moc zbioru: wysoka
Liczba etykiet zmiennej home.dest: 370, Moc zbioru: wysoka


2. Wyświetl z użyciem funkcji print liczbę wszystkich pasażerów. Wykorzystaj podobny sposób jak w ptk 1. 

In [7]:
print(f'Liczba pasażerów {len(df)}')

Liczba pasażerów 1309


3.     Skomentuj wyniki otrzymane w punkcie 1 i 2. Podziel zmienne ze względu na dużą i małą moc zbioru (kardynalność). 

In [8]:
for column_name in categorical_columns:
    unique_values = len(df[f'{column_name}'].unique())
    # Uznałem 10 jako granicę mocy zbioru pomiędzy wysoką a niską
    cardinality = 'wysoka' if unique_values > 10 else 'niska'
    print(f'Moc zbioru zmiennej {column_name}: {cardinality}')

Moc zbioru zmiennej pclass: niska
Moc zbioru zmiennej survived: niska
Moc zbioru zmiennej name: wysoka
Moc zbioru zmiennej sex: niska
Moc zbioru zmiennej sibsp: niska
Moc zbioru zmiennej parch: niska
Moc zbioru zmiennej ticket: wysoka
Moc zbioru zmiennej cabin: wysoka
Moc zbioru zmiennej embarked: niska
Moc zbioru zmiennej boat: wysoka
Moc zbioru zmiennej body: wysoka
Moc zbioru zmiennej home.dest: wysoka


Komentarz:
 - Wszystkie zmienne poniżej 10 uznałem za mające niską moc zbioru, ze względu na relatwyną moc do pozostałych, gdzie wartości są > 100
 - Jedyna zmienna posiadająca 'średnią' moc zbioru to boat, ale ze względu na narzucony podział uznałem ją za wysoką

4. Sprawdź, ile unikalnych etykiet ma zmienna mówiąca o kabinie danego pasażera. Użyj takiej funkcji, która zwraca wynik w postaci NumPy array.

In [9]:
cabin_unique = df.cabin.unique()
print(cabin_unique)
print("Typ:",type(cabin_unique))

['B5' 'C22 C26' 'E12' 'D7' 'A36' 'C101' None 'C62 C64' 'B35' 'A23'
 'B58 B60' 'D15' 'C6' 'D35' 'C148' 'C97' 'B49' 'C99' 'C52' 'T' 'A31' 'C7'
 'C103' 'D22' 'E33' 'A21' 'B10' 'B4' 'E40' 'B38' 'E24' 'B51 B53 B55'
 'B96 B98' 'C46' 'E31' 'E8' 'B61' 'B77' 'A9' 'C89' 'A14' 'E58' 'E49' 'E52'
 'E45' 'B22' 'B26' 'C85' 'E17' 'B71' 'B20' 'A34' 'C86' 'A16' 'A20' 'A18'
 'C54' 'C45' 'D20' 'A29' 'C95' 'E25' 'C111' 'C23 C25 C27' 'E36' 'D34'
 'D40' 'B39' 'B41' 'B102' 'C123' 'E63' 'C130' 'B86' 'C92' 'A5' 'C51' 'B42'
 'C91' 'C125' 'D10 D12' 'B82 B84' 'E50' 'D33' 'C83' 'B94' 'D49' 'D45'
 'B69' 'B11' 'E46' 'C39' 'B18' 'D11' 'C93' 'B28' 'C49' 'B52 B54 B56' 'E60'
 'C132' 'B37' 'D21' 'D19' 'C124' 'D17' 'B101' 'D28' 'D6' 'D9' 'B80' 'C106'
 'B79' 'C47' 'D30' 'C90' 'E38' 'C78' 'C30' 'C118' 'D36' 'D48' 'D47' 'C105'
 'B36' 'B30' 'D43' 'B24' 'C2' 'C65' 'B73' 'C104' 'C110' 'C50' 'B3' 'A24'
 'A32' 'A11' 'A10' 'B57 B59 B63 B66' 'C28' 'E44' 'A26' 'A6' 'A7' 'C31'
 'A19' 'B45' 'E34' 'B78' 'B50' 'C87' 'C116' 'C55 C57' 'D50

5.     Zredukuj liczbę cech dla zmiennej opisującej kabiny poprzez zastąpienie obecnych etykiet w formacie LL11 do etykiet zawierających tylko pierwszą literę. Użyj astype(str).str[pozycja]. Nową zmienną nazwij CabinReduced. Wyświetl pierwsze 20 wierszy zbioru danych dla kolumn Cabin i CabinReduced

In [10]:
df['CabinReduced'] = df['cabin'].astype(str).str[0]
df[['CabinReduced','cabin']].head(20)

,CabinReduced,cabin
0,B,B5
1,C,C22 C26
2,C,C22 C26
3,C,C22 C26
4,C,C22 C26
5,E,E12
6,D,D7
7,A,A36
8,C,C101
9,N,None


6.     Wyświetl (jak w pkt 1) liczbę etykiet dla zmiennych z ptk 5. O ile procent zredukowano kardynalność zbioru zmiennej opisującej kabiny?

In [11]:
unique_values_red = len(df['CabinReduced'].unique())
cardinality = 'wysoka' if unique_values > 10 else 'niska'
print(f'Liczba etykiet zmiennej CabinReduced: {unique_values_red}, Moc zbioru: {cardinality}')
unique_values = len(df['cabin'].unique())
cardinality = 'wysoka' if unique_values > 10 else 'niska'
print(f'Liczba etykiet zmiennej cabin: {unique_values}, Moc zbioru: {cardinality}')
procent_redukcji = (unique_values-unique_values_red)/unique_values * 100
print(f'Kardynalność zbioru zredukowano o ',"{:.2f}".format(procent_redukcji), "%")

Liczba etykiet zmiennej CabinReduced: 9, Moc zbioru: wysoka
Liczba etykiet zmiennej cabin: 187, Moc zbioru: wysoka
Kardynalność zbioru zredukowano o  95.19 %


7.     Uzasadnij dlaczego dokonujesz redukcji akurat tej zmiennej. Jak to wpływa na przyszłe analizy. Czy powoduje jakieś negatywne skutki?

- Ponieważ, kolumna cabin, była cechą o bardzo wysokiej kardynalności, zmniejeszenie jej ze 187 do 9, pozwala nam na łatwiejsze przetwarzanie i interpretację jej.
- Odpowiadając, na pytanie czy powoduje to jakieś negatwyne skutki ? to zależy.
- Pozbywamy się informacji o konkretnej kabinie, lecz może to być informacja nieistotna w zależności od natury dalszej analizy.
- Trudno jest na tym etapie jednoznacznie ocenić, czy redukcja będzie miała negatywne czy pozytywne skutki.

In [14]:
df.to_csv('dane.csv',index=False)